In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import boto3
import aws_util
import cfg
import yaml
import ray
from pathlib import Path

import regroup
import ray_util
import util
import process_commit_pairs
import render

## Ray cluster management 

In [3]:
import ray_server
server = ray_server.get_ray_server()

2024-02-27 19:21:33,231	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 10.210.154.158:8786...
2024-02-27 19:21:33,262	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at http://10.210.154.158:8000 


https://453f3e01-e179-4204-a29b-3eb6bb94a853-8000.job.console.elementai.com


In [4]:
server.scale_cluster(60)

'OK'

## Load secrets

In [5]:
with open('secrets.yaml') as f:
    secrets = yaml.safe_load(f)

## Get permissive licenses and convert them to sqlite

In [6]:
aws_util.download_s3._function(
    cfg.repo_licenses_s3['bucket'],
    [cfg.repo_licenses_s3['path']],
    cfg.repo_licenses_path,
    secrets['aws_access_key_id'],
    secrets['aws_secret_access_key']
)

In [ ]:
util.repo_names_licenses_convert_to_sqlite(
    cfg.repo_licenses_path / Path(cfg.repo_licenses_s3['path']).name,
    cfg.repo_licenses_sqlite_file
)

2024-02-27 20:06:57,855	WARNING worker.py:2058 -- The node with node id: eb6161373dd810eb1c6006acce4ba31c2b4cdb21c55e1d1fcdd4f6cb and address: 10.210.142.8 and node name: 10.210.142.8 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.
2024-02-27 20:09:21,012	WARNING worker.py:2058 -- The node with node id: 28835ff69e9f4134cdda660fa1096ee90823f4cd0de1a6e955788c6c and address: 10.210.174.138 and node name: 10.210.174.138 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.
2024-02-27 20:16:55,405	WARNING worker.py:2058 -- The node with node id: 78ddbe7cece04c64f1b371c5c91410545212c1f8192be1b401e4ed5d and address: 10.21

## Download commit pairs

In [ ]:
file_list = aws_util.list_parquet_files_s3(
    cfg.commit_paris_files_s3['bucket'],
    cfg.commit_paris_files_s3['path'],
    secrets['aws_access_key_id'],
    secrets['aws_secret_access_key']
)

In [ ]:
file_groups = regroup.split_items(file_list, 10, False)

In [ ]:
res = []
for gr in file_groups[1:]:
    res.append(aws_util.download_s3.remote(
        cfg.commit_paris_files_s3['bucket'],
        gr,
        cfg.pr_commit_pairs_files_path,
        secrets['aws_access_key_id'],
        secrets['aws_secret_access_key']
    ))

In [ ]:
ray_util.ray_tasks_progress(res)
res = ray.get(res)

## Get opt outs

In [ ]:
repos_opt_out, users_for_repo_opt_out, users_for_commits_opt_out, users_for_issues_opt_out = util.get_opt_outs(
    src=cfg.opt_outs_dataset_name,
    token=secrets['hf_api_key']
)

## Filter opt outs and non permissive licenses, get range of changed for each file pair, remove new or deleted files 

In [ ]:
files = list(cfg.pr_commit_pairs_files_path.glob('*.parquet'))
dst = cfg.pr_commid_pairs_files_filtered_path
dst.mkdir(parents=True, exist_ok=True)
max_changes_length = render.RenderParams().max_pr_length

In [ ]:
res = []
for f in files:
    res.append(process_commit_pairs.filter_nonpermissive_opt_outs_and_prepare_commit_pairs.remote(
        f, dst, repos_opt_out, users_for_repo_opt_out, max_changes_length, cfg.repo_licenses_sqlite_file
    ))

In [ ]:
ray_util.ray_tasks_progress(res)
res = ray.get(res)

## Filter files by content

In [ ]:
files = list(cfg.pr_commid_pairs_files_filtered_path.glob('*.parquet'))
dst  = cfg.pr_commid_pairs_files_filtered_cleaned_path
dst.mkdir(parents=True, exist_ok=True)

In [ ]:
params = process_commit_pairs.FilterParams()
res = util.ray_map(
    process_commit_pairs.clean_files_bucket,
    files,
    dst=dst,
    filter_params=params
)


In [ ]:
ray_util.ray_tasks_progress(res)
res = ray.get(res)

In [ ]:
res = regroup.ray_shuffle.remote(
    cfg.pr_commid_pairs_files_filtered_cleaned_path,
    cfg.pr_commid_pairs_files_filtered_cleaned_grouped_path',
    'pull_request.guid',
    3
)

In [ ]:
res = ray.get(res)

## Ray cluster management

In [ ]:
ray.shutdown()
server.scale_cluster(0)